## Imports

In [1]:
import pandas as pd
import numpy as np
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)
torch.get_default_device()

device(type='cuda', index=0)

In [8]:
r = pd.read_csv("./results/summarized/1731930763_models_train_test_20.csv", 
                index_col="id", 
                usecols=['id','level', 'batch_size', 'epochs','model', 'learning_rate', 'error', 'best_epoch', 'train_acc_best_epoch','test_acc_best_epoch']
                )
r = r.loc[r["error"].isna()].drop(columns=["error"]).sort_values(by=["level", "test_acc_best_epoch"], ascending=[True, False])
r

,level,batch_size,epochs,model,learning_rate,best_epoch,train_acc_best_epoch,test_acc_best_epoch
id,,,,,,,,
16,class,dynamic,150,SimplestCNNClassifier,0.0005,122,0.999928,0.934165
19,class,dynamic,150,UnetBasedCNNWithDropoutClassifier,0.0005,108,0.999370,0.888817
18,class,dynamic,150,UnetBasedCNNWithDilationClassifier,0.0005,122,0.999955,0.878209
17,class,dynamic,150,UnetBasedCNNClassifier,0.0005,135,0.999937,0.870354
12,family,dynamic,150,SimplestCNNClassifier,0.0005,149,0.999892,0.901147
14,family,dynamic,150,UnetBasedCNNWithDilationClassifier,0.0005,139,0.999959,0.835733
15,family,dynamic,150,UnetBasedCNNWithDropoutClassifier,0.0005,130,0.997735,0.818404
13,family,dynamic,150,UnetBasedCNNClassifier,0.0005,145,0.999973,0.809373
0,genus,dynamic,150,SimplestCNNClassifier,0.0005,134,0.998401,0.746702


In [9]:
a = pd.read_csv("./results/summarized/1732414050_models_train_test_5.csv", 
                index_col="id", 
                usecols=['id','level', 'batch_size', 'epochs','model', 'learning_rate', 'error', 'best_epoch', 'train_acc_best_epoch','test_acc_best_epoch']
                )
a = a.loc[a["error"].isna()].drop(columns=["error"]).sort_values(by=["level", "test_acc_best_epoch"], ascending=[True, False])
a

,level,batch_size,epochs,model,learning_rate,best_epoch,train_acc_best_epoch,test_acc_best_epoch
id,,,,,,,,
0,class,dynamic,400,SimpleCNNClassifier1,0.0005,153,0.999883,0.932950
2,family,dynamic,400,SimpleCNNClassifier1,0.0005,69,0.999553,0.878936
3,genus,dynamic,400,SimpleCNNClassifier1,0.0005,77,0.993922,0.726793
4,order,dynamic,400,SimpleCNNClassifier1,0.0005,272,0.999768,0.900865
1,species,dynamic,400,SimpleCNNClassifier1,0.0005,77,0.991714,0.710368


In [10]:
pd.concat([r,a]).sort_values(by=["level", "test_acc_best_epoch"], ascending=[True, False])

,level,batch_size,epochs,model,learning_rate,best_epoch,train_acc_best_epoch,test_acc_best_epoch
id,,,,,,,,
16,class,dynamic,150,SimplestCNNClassifier,0.0005,122,0.999928,0.934165
0,class,dynamic,400,SimpleCNNClassifier1,0.0005,153,0.999883,0.932950
19,class,dynamic,150,UnetBasedCNNWithDropoutClassifier,0.0005,108,0.999370,0.888817
18,class,dynamic,150,UnetBasedCNNWithDilationClassifier,0.0005,122,0.999955,0.878209
17,class,dynamic,150,UnetBasedCNNClassifier,0.0005,135,0.999937,0.870354
12,family,dynamic,150,SimplestCNNClassifier,0.0005,149,0.999892,0.901147
2,family,dynamic,400,SimpleCNNClassifier1,0.0005,69,0.999553,0.878936
14,family,dynamic,150,UnetBasedCNNWithDilationClassifier,0.0005,139,0.999959,0.835733
15,family,dynamic,150,UnetBasedCNNWithDropoutClassifier,0.0005,130,0.997735,0.818404


In [11]:
r.columns

Index(['level', 'batch_size', 'epochs', 'model', 'learning_rate', 'best_epoch',
       'train_acc_best_epoch', 'test_acc_best_epoch'],
      dtype='object')

## Data Load

In [ ]:
base_map = {
    "A":[1.0, 0.0, 0.0, 0.0],
    "T":[0.0, 1.0, 0.0, 0.0],
    "G":[0.0, 0.0, 1.0, 0.0],
    "C":[0.0, 0.0, 0.0, 1.0],

    'W':[0.5, 0.5, 0.0, 0.0],
    'S':[0.0, 0.0, 0.5, 0.5],
    'M':[0.5, 0.0, 0.0, 0.5],
    'K':[0.0, 0.5, 0.5, 0.0],
    'R':[0.5, 0.0, 0.5, 0.0],
    'Y':[0.0, 0.5, 0.0, 0.5],
    
    'B':[0.0, 0.3, 0.3, 0.3],
    'D':[0.3, 0.3, 0.3, 0.0],
    'H':[0.3, 0.3, 0.0, 0.3],
    'V':[0.3, 0.0, 0.3, 0.3],

    'N':[0.25, 0.25, 0.25, 0.25],
}

def encode_sequence(sequence):
    encoded_seq = []

    for base in sequence:
        encoded_seq.append(base_map[base])
    
    return torch.tensor(encoded_seq)

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, train, test, level):

        self.classes = pd.concat([train[level], test[level]]).unique().tolist()
        self.classes.sort()
        self.level = level
        self.labels = train[level]
        self.encoded_labels = SequenceDataset.__encoded_labels__(self.classes, self.labels)
        self.sequences = SequenceDataset.__sequences__(train)

        self.test = SequenceDatasetTest(
            labels = test[level],
            classes = self.classes,
            encoded_labels = SequenceDataset.__encoded_labels__(self.classes, test[level]),
            sequences = SequenceDataset.__sequences__(test)
            )

    def __encoded_labels__(classes, labels):
        return torch.nn.functional.one_hot(torch.tensor([classes.index(l) for l in labels]), len(classes)).type(torch.cuda.FloatTensor)
    
    def __sequences__(ds):
        sequences = []
        for _, row in ds.iterrows():
            sequences.append(encode_sequence(row["truncated_sequence"]))        
        return torch.stack(sequences, dim=0)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return   self.sequences[idx], self.encoded_labels[idx]
    
    def __getitems__(self, ids):
        idx = torch.tensor(ids, device=torch.device('cuda:0'))
        return   list(zip(torch.index_select(self.sequences, 0, idx), torch.index_select(self.encoded_labels, 0, idx)))
    
    def get_test(self):
        return self.test

class SequenceDatasetTest(SequenceDataset):    
    def __init__(self, labels, classes, encoded_labels, sequences):
        self.labels = labels
        self.classes = classes
        self.encoded_labels = encoded_labels
        self.sequences = sequences

In [ ]:
levels = ["domain", "class", "order", "family", "genus", "species"]

class MultilevelSequenceDataset(Dataset):
    def __init__(self, train, test, level):

        self.classes = pd.concat([train[level], test[level]]).unique().tolist()
        self.classes.sort()
        self.level = level

        self.labels = train[level]
        self.encoded_labels = MultilevelSequenceDataset.__encoded_labels__(self.classes, self.labels)
        self.sequences = MultilevelSequenceDataset.__sequences__(train)        

        self.previous_level = levels[levels.index(level)-1]
        self.previous_classes = pd.concat([train[self.previous_level], test[self.previous_level]]).unique().tolist()
        self.previous_classes.sort()
        self.previous_encoded_labels = MultilevelSequenceDataset.__encoded_labels__(self.previous_classes, train[self.previous_level])


        self.test = MultilevelSequenceDatasetTest(
            labels = test[level],
            classes = self.classes,
            encoded_labels = MultilevelSequenceDataset.__encoded_labels__(self.classes, test[level]),
            sequences = MultilevelSequenceDataset.__sequences__(test),
            previous_classes = self.previous_classes,
            previous_encoded_labels = MultilevelSequenceDataset.__encoded_labels__(self.previous_classes, test[self.previous_level]) ,
            previous_level = self.previous_level
            )

    def __encoded_labels__(classes, labels):
        return torch.nn.functional.one_hot(torch.tensor([classes.index(l) for l in labels]), len(classes)).type(torch.cuda.FloatTensor)
    
    def __sequences__(ds):
        sequences = []
        for _, row in ds.iterrows():
            sequences.append(encode_sequence(row["truncated_sequence"]))        
        return torch.stack(sequences, dim=0)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return   self.sequences[idx], self.previous_encoded_labels[idx], self.encoded_labels[idx]
    
    def __getitems__(self, ids):
        idx = torch.tensor(ids, device=torch.device('cuda:0'))
        return   list(zip(torch.index_select(self.sequences, 0, idx), torch.index_select(self.previous_encoded_labels, 0, idx), torch.index_select(self.encoded_labels, 0, idx)))
    
    def get_test(self):
        return self.test

class MultilevelSequenceDatasetTest(MultilevelSequenceDataset):    
    def __init__(self, labels, classes, encoded_labels, sequences, previous_level, previous_classes, previous_encoded_labels ):
        self.labels = labels
        self.classes = classes
        self.encoded_labels = encoded_labels
        self.sequences = sequences

        self.previous_level = previous_level 
        self.previous_classes = previous_classes
        self.previous_encoded_labels = previous_encoded_labels

In [ ]:
def loaders_generator(ds_train, ds_test, bs = 128):
    train_loader = DataLoader(ds_train, batch_size=bs, shuffle=True, generator=torch.Generator(device='cuda'))
    test_loader = DataLoader(ds_test, batch_size=bs, shuffle=True, generator=torch.Generator(device='cuda'))

    return train_loader, test_loader

## Models

## Single Level

In [ ]:
class SimplestCNNClassifier(nn.Module):
    
    def __init__(self, nClasses):
        super(SimplestCNNClassifier, self).__init__()

        self.padding1 = nn.CircularPad1d((1,2))
        self.conv1 = nn.Conv1d(4, 8, kernel_size=4)
        self.adAvgPool1 = nn.AdaptiveAvgPool1d(450)

        self.padding2 = nn.CircularPad1d((1,2))
        self.conv2 = nn.Conv1d(8, 32, kernel_size=4)
        self.adAvgPool2 = nn.AdaptiveAvgPool1d(225)

        self.padding3 = nn.CircularPad1d((1,2))
        self.conv3 = nn.Conv1d(32, 128, kernel_size=4)
        self.adAvgPool3 = nn.AdaptiveAvgPool1d(225)


        self.act4 = nn.ReLU()

        self.linear1 = nn.Linear(28800, 28800*2)
        self.linear2 = nn.Linear(28800*2, nClasses)
    
    def forward(self, x):

        x = torch.movedim(x, -1, -2)

        x = self.conv1(self.padding1(x))
        x = self.adAvgPool1(x)

        
        x = self.conv2(self.padding2(x))
        x = self.adAvgPool2(x)

        
        x = self.conv3(self.padding3(x))
        x = self.adAvgPool3(x)
        
        
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = self.act4(x)
        x = self.linear2(x)

        return x

In [ ]:

class SimpleCNNClassifier(nn.Module):
    
    def __init__(self, nClasses):
        super(SimpleCNNClassifier, self).__init__()

        self.padding = nn.CircularPad1d((1,2))
        
        self.conv1 = nn.Conv1d(4, 4, kernel_size=4, groups=4)
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv1d(4, 4, kernel_size=4)        
        self.act2 = nn.ReLU()
        
        self.act3 = nn.ReLU()        
        self.conv3 = nn.Conv1d(4, 8, kernel_size=4, groups=4, dilation=2, padding=3, padding_mode="circular")
        
        self.act4 = nn.ReLU()

        # self.adAvgPool = nn.AdaptiveAvgPool1d(900)

        self.linear1 = nn.Linear(14400, 28800)
        self.linear2 = nn.Linear(28800, nClasses)
    
    def forward(self, x):

        x = torch.movedim(x, -1, -2)
        a = self.conv1(self.padding(x))
        a = self.act1(a)        
        
        b = self.conv2(self.padding(x))
        b = self.act2(b)
        
        c = self.conv3(x)
        c = self.act3(c)
        
        x = torch.flatten(torch.cat([a,b,c], dim=1), 1)
        
        x = self.linear1(x)
        
        x = self.act4(x)
        
        x = self.linear2(x)

        return x

In [ ]:

class SimpleCNNWithDropoutClassifier(nn.Module):
    
    def __init__(self, nClasses):
        super(SimpleCNNWithDropoutClassifier, self).__init__()

        self.padding = nn.CircularPad1d((1,2))

        self.dropout1 = nn.Dropout(p=0.2)
        self.conv1 = nn.Conv1d(4, 4, kernel_size=4, groups=4)
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv1d(4, 4, kernel_size=4)
        self.act2 = nn.ReLU()
        
        self.act3 = nn.ReLU()
        self.conv3 = nn.Conv1d(4, 8, kernel_size=4, groups=4, dilation=2, padding=3, padding_mode="circular")

        # self.adAvgPool = nn.AdaptiveAvgPool1d(900)

        self.act4 = nn.ReLU()

        self.dropout2 = nn.Dropout(p=0.2)
        self.linear1 = nn.Linear(14400, 28800)

        self.dropout3 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(28800, nClasses)
    
    def forward(self, x):

        x = torch.movedim(x, -1, -2)
        x = self.dropout1(x)
        a = self.conv1(self.padding(x))
        a = self.act1(a)
        
        b = self.conv2(self.padding(x))
        b = self.act2(b)
        
        c = self.conv3(x)
        c = self.act3(c)
        
        x = torch.flatten(torch.cat([a,b,c], dim=1), 1)
        x = self.dropout2(x)
        
        x = self.linear1(x)
        
        x = self.act4(x)
        x = self.dropout3(x)
        
        x = self.linear2(x)

        return x

In [ ]:

class BaseCNNClassifier(nn.Module):
    
    def __init__(self, nClasses):
        super(BaseCNNClassifier, self).__init__()

        self.conv1_1 = nn.Conv1d(1, 4, kernel_size=4)
        self.conv1_2 = nn.Conv1d(1, 4, kernel_size=4, dilation=2)
        self.avgPool = nn.AvgPool1d(4, stride=2)
        
        self.padding = nn.CircularPad1d((1,2))
        
        self.act1 = nn.ReLU()
        self.act2 = nn.ReLU()

        self.linear1 = nn.Linear(14392, 14392)
        self.linear2 = nn.Linear(14392, nClasses)
    
    def forward(self, x):

        x = torch.unsqueeze(torch.flatten(x, start_dim=1), 1)
        x = self.padding(x)
        
        x_1_1 = self.conv1_1(x)
        x_1_2 = self.conv1_2(self.padding(x))      

        x = torch.cat([x_1_1, x_1_2], dim=1)
        x = self.avgPool(x)

        x = torch.flatten(x, 1)
        
        x = self.act1(x)
        x = self.linear1(x)
        
        x = self.act2(x)
        x = self.linear2(x)
        
        return x

In [ ]:

class UnetBasedCNNClassifier(nn.Module):
    
    def __init__(self, nClasses):
        super(UnetBasedCNNClassifier, self).__init__()

        # First Encode Level
        self.padding_e_1_1 = nn.CircularPad1d((1,2))
        self.conv_e_1_1 = nn.Conv1d(4, 8, kernel_size=4)
        self.act_e_1_1 = nn.ReLU()

        self.padding_e_1_2 = nn.CircularPad1d((1,2))
        self.conv_e_1_2 = nn.Conv1d(8, 8, kernel_size=4)
        self.act_e_1_2 = nn.ReLU()

        self.avgPool_e_1_1 = nn.AvgPool1d(2, stride=2)


        # Second Encode Level        
        self.padding_e_2_1 = nn.CircularPad1d((1,2))
        self.conv_e_2_1 = nn.Conv1d(8, 16, kernel_size=4)
        self.act_e_2_1 = nn.ReLU()
        
        self.padding_e_2_2 = nn.CircularPad1d((1,2))
        self.conv_e_2_2 = nn.Conv1d(16, 16, kernel_size=4)
        self.act_e_2_2 = nn.ReLU()

        self.avgPool_e_2_1 = nn.AvgPool1d(2, stride=2)

        
        # Transition Level
        self.padding_t_1_1 = nn.CircularPad1d((1,2))
        self.conv_t_1_1 = nn.Conv1d(16, 32, kernel_size = 4)
        self.act_t_1_1 = nn.ReLU()
        
        self.padding_t_1_2 = nn.CircularPad1d((1,2))
        self.conv_t_1_2 = nn.Conv1d(32, 32, kernel_size = 4)
        self.act_t_1_2 = nn.ReLU()


        # First Decode Level
        self.upconv_1_1 = nn.ConvTranspose1d(32, 16, kernel_size=2, stride=2)

        self.padding_d_1_1 = nn.CircularPad1d((1,2))
        self.conv_d_1_1 = nn.Conv1d(32, 16, kernel_size=4)
        self.act_d_1_1 = nn.ReLU()

        self.padding_d_1_2 = nn.CircularPad1d((1,2))
        self.conv_d_1_2 = nn.Conv1d(16, 16, kernel_size=4)
        self.act_d_1_2 = nn.ReLU()


        # Second Decode Level
        self.upconv_2_1 = nn.ConvTranspose1d(16, 8, kernel_size=2, stride=2)

        self.padding_d_2_1 = nn.CircularPad1d((1,2))
        self.conv_d_2_1 = nn.Conv1d(16, 8, kernel_size=4)
        self.act_d_2_1 = nn.ReLU()

        self.padding_d_2_2 = nn.CircularPad1d((1,2))
        self.conv_d_2_2 = nn.Conv1d(8, 8, kernel_size=4)
        self.act_d_2_2 = nn.ReLU()


        # Output Level
        self.conv_out_1 = nn.Conv1d(8, nClasses, kernel_size=1)
        
        self.linear_out_1 = nn.Linear(7200, 14400)
        self.act_out_1 = nn.ReLU()
        self.linear_out_2 = nn.Linear(14400, nClasses)
        self.act_out_2 = nn.ReLU()
    
    def forward(self, x):

        # print("X shape: "+str(x.shape))             # X shape: torch.Size([3, 900, 4])

        x = x.reshape(x.shape[0],4,-1)              # Reshape X shape: torch.Size([3, 4, 900])
        # print("Reshape X shape: "+str(x.shape))
        # print("\n")


        # print("\n------------E 1-------------")
        x_e_1 = self.padding_e_1_1(x)               # Padding X shape: torch.Size([3, 4, 900])
        # print("Padding X shape: "+str(x.shape))
        x_e_1 = self.conv_e_1_1(x_e_1)              # Conv X shape: torch.Size([3, 8, 900])
        # print("Conv X shape: "+str(x_e_1.shape))
        x_e_1 = self.act_e_1_1(x_e_1)               # ActFunc X shape: torch.Size([3, 8, 900])
        # print("ActFunc X shape: "+str(x_e_1.shape))
        
        x_e_1 = self.padding_e_1_2(x_e_1)           # Padding X shape: torch.Size([3, 8, 903])
        # print("Padding X shape: "+str(x_e_1.shape))
        x_e_1 = self.conv_e_1_2(x_e_1)              # Conv X shape: torch.Size([3, 8, 900])
        # print("Conv X shape: "+str(x_e_1.shape))
        x_e_1 = self.act_e_1_2(x_e_1)               # ActFunc X shape: torch.Size([3, 8, 900])
        # print("ActFunc X shape: "+str(x_e_1.shape))
        
        x_e_p_1 = self.avgPool_e_1_1(x_e_1)         # Pool X shape: torch.Size([3, 8, 450])
        # print("Pool X shape: "+str(x_e_p_1.shape))


        # print("\n-----------E 2--------------")
        x_e_2 = self.padding_e_2_1(x_e_p_1)         # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_e_2.shape))
        x_e_2 = self.conv_e_2_1(x_e_2)              # Conv X shape: torch.Size([3, 16, 450])
        # print("Conv X shape: "+str(x_e_2.shape))
        x_e_2 = self.act_e_2_1(x_e_2)               # ActFunc X shape: torch.Size([3, 16, 450])
        # print("ActFunc X shape: "+str(x_e_2.shape))
        
        x_e_2 = self.padding_e_2_2(x_e_2)           # Padding X shape: torch.Size([3, 16, 453])
        # print("Padding X shape: "+str(x_e_2.shape))
        x_e_2 = self.conv_e_2_2(x_e_2)              # Conv X shape: torch.Size([3, 16, 450])
        # print("Conv X shape: "+str(x_e_2.shape))
        x_e_2 = self.act_e_2_2(x_e_2)               # ActFunc X shape: torch.Size([3, 16, 450])
        # print("ActFunc X shape: "+str(x_e_2.shape))

        x_e_p_2 = self.avgPool_e_2_1(x_e_2)         # Pool X shape: torch.Size([3, 16, 225])
        # print("Pool X shape: "+str(x_e_p_2.shape))


        # print("\n------------T 1-------------")
        x_t_1 = self.padding_t_1_1(x_e_p_2)         # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_t_1.shape))
        x_t_1 = self.conv_t_1_1(x_t_1)              # Conv X shape: torch.Size([3, 32, 109])
        # print("Conv X shape: "+str(x_t_1.shape))
        x_t_1 = self.act_t_1_1(x_t_1)               # ActFunc X shape: torch.Size([3, 32, 109])
        # print("ActFunc X shape: "+str(x_t_1.shape))
        
        x_t_1 = self.padding_t_1_2(x_t_1)           # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_t_1.shape))
        x_t_1 = self.conv_t_1_2(x_t_1)              # Conv X shape: torch.Size([3, 32, 106])
        # print("Conv X shape: "+str(x_t_1.shape))
        x_t_1 = self.act_t_1_2(x_t_1)               # ActFunc X shape: torch.Size([3, 32, 106])
        # print("ActFunc X shape: "+str(x_t_1.shape))


        # print("\n------------D 1------------")
        x_d_1 = self.upconv_1_1(x_t_1)              # UpConv X shape: torch.Size([3, 16, 214])
        # print("UpConv X shape: "+str(x_d_1.shape))

        x_d_1 = torch.cat([x_d_1, x_e_2], dim=1)    # 
        # print("Cat X shape: "+str(x_d_1.shape))

        x_d_1 = self.padding_e_1_1(x_d_1)           # 
        # print("Padding X shape: "+str(x_d_1.shape))
        x_d_1 = self.conv_d_1_1(x_d_1)              # 
        # print("Conv X shape: "+str(x_d_1.shape))
        x_d_1 = self.act_d_1_1(x_d_1)               # 
        # print("ActFunc X shape: "+str(x_d_1.shape))

        x_d_1 = self.padding_e_1_1(x_d_1)           # 
        # print("Padding X shape: "+str(x_d_1.shape))
        x_d_1 = self.conv_d_1_2(x_d_1)              # 
        # print("Conv X shape: "+str(x_d_1.shape))
        x_d_1 = self.act_d_1_2(x_d_1)               # 
        # print("ActFunc X shape: "+str(x_d_1.shape))


        # print("\n------------D 2------------")
        x_d_2 = self.upconv_2_1(x_d_1)              # UpConv X shape: torch.Size([3, 16, 214])
        # print("UpConv X shape: "+str(x_d_2.shape))

        x_d_2 = torch.cat([x_d_2, x_e_1], dim=1)    # 
        # print("Cat X shape: "+str(x_d_2.shape))

        x_d_2 = self.padding_e_1_1(x_d_2)           # 
        # print("Padding X shape: "+str(x_d_2.shape))
        x_d_2 = self.conv_d_2_1(x_d_2)              # 
        # print("Conv X shape: "+str(x_d_2.shape))
        x_d_2 = self.act_d_2_1(x_d_2)               # 
        # print("ActFunc X shape: "+str(x_d_2.shape))

        x_d_2 = self.padding_e_1_1(x_d_2)           # 
        # print("Padding X shape: "+str(x_d_2.shape))
        x_d_2 = self.conv_d_2_2(x_d_2)              # 
        # print("Conv X shape: "+str(x_d_2.shape))
        x_d_2 = self.act_d_2_2(x_d_2)               # 
        # print("ActFunc X shape: "+str(x_d_2.shape))


        # print("\n------------Out------------")
        # x = self.conv_out_1(x_d_2)


        x = torch.flatten(x_d_2, 1)
        
        x = self.act_out_1(x)
        x = self.linear_out_1(x)
        
        x = self.act_out_2(x)
        x = self.linear_out_2(x)

        # print("Output X shape: "+str(x.shape))

        return x

In [ ]:

class UnetBasedCNNWithDropoutClassifier(nn.Module):
    
    def __init__(self, nClasses):
        super(UnetBasedCNNWithDropoutClassifier, self).__init__()

        self.input_dropout1 = nn.Dropout(p=0.2)

        # First Encode Level
        self.padding_e_1_1 = nn.CircularPad1d((1,2))
        self.conv_e_1_1 = nn.Conv1d(4, 8, kernel_size=4)
        self.act_e_1_1 = nn.ReLU()

        self.padding_e_1_2 = nn.CircularPad1d((1,2))
        self.conv_e_1_2 = nn.Conv1d(8, 8, kernel_size=4)
        self.act_e_1_2 = nn.ReLU()

        self.avgPool_e_1_1 = nn.AvgPool1d(2, stride=2)


        # Second Encode Level        
        self.padding_e_2_1 = nn.CircularPad1d((1,2))
        self.conv_e_2_1 = nn.Conv1d(8, 16, kernel_size=4)
        self.act_e_2_1 = nn.ReLU()
        
        self.padding_e_2_2 = nn.CircularPad1d((1,2))
        self.conv_e_2_2 = nn.Conv1d(16, 16, kernel_size=4)
        self.act_e_2_2 = nn.ReLU()

        self.avgPool_e_2_1 = nn.AvgPool1d(2, stride=2)

        
        # Transition Level
        self.padding_t_1_1 = nn.CircularPad1d((1,2))
        self.conv_t_1_1 = nn.Conv1d(16, 32, kernel_size = 4)
        self.act_t_1_1 = nn.ReLU()
        
        self.padding_t_1_2 = nn.CircularPad1d((1,2))
        self.conv_t_1_2 = nn.Conv1d(32, 32, kernel_size = 4)
        self.act_t_1_2 = nn.ReLU()


        # First Decode Level
        self.upconv_1_1 = nn.ConvTranspose1d(32, 16, kernel_size=2, stride=2)

        self.padding_d_1_1 = nn.CircularPad1d((1,2))
        self.conv_d_1_1 = nn.Conv1d(32, 16, kernel_size=4)
        self.act_d_1_1 = nn.ReLU()

        self.padding_d_1_2 = nn.CircularPad1d((1,2))
        self.conv_d_1_2 = nn.Conv1d(16, 16, kernel_size=4)
        self.act_d_1_2 = nn.ReLU()


        # Second Decode Level
        self.upconv_2_1 = nn.ConvTranspose1d(16, 8, kernel_size=2, stride=2)

        self.padding_d_2_1 = nn.CircularPad1d((1,2))
        self.conv_d_2_1 = nn.Conv1d(16, 8, kernel_size=4)
        self.act_d_2_1 = nn.ReLU()

        self.padding_d_2_2 = nn.CircularPad1d((1,2))
        self.conv_d_2_2 = nn.Conv1d(8, 8, kernel_size=4)
        self.act_d_2_2 = nn.ReLU()


        # Output Level
        self.conv_out_1 = nn.Conv1d(8, nClasses, kernel_size=1)
        
        
        self.output_dropout1 = nn.Dropout(p=0.2)
        self.linear_out_1 = nn.Linear(7200, 14400)
        self.act_out_1 = nn.ReLU()

        
        self.output_dropout2 = nn.Dropout(p=0.2)
        self.linear_out_2 = nn.Linear(14400, nClasses)
        self.act_out_2 = nn.ReLU()
    
    def forward(self, x):

        # print("X shape: "+str(x.shape))             # X shape: torch.Size([3, 900, 4])

        x = x.reshape(x.shape[0],4,-1)              # Reshape X shape: torch.Size([3, 4, 900])
        # print("Reshape X shape: "+str(x.shape))
        # print("\n")

        x = self.input_dropout1(x)

        # print("\n------------E 1-------------")
        x_e_1 = self.padding_e_1_1(x)               # Padding X shape: torch.Size([3, 4, 900])
        # print("Padding X shape: "+str(x.shape))
        x_e_1 = self.conv_e_1_1(x_e_1)              # Conv X shape: torch.Size([3, 8, 900])
        # print("Conv X shape: "+str(x_e_1.shape))
        x_e_1 = self.act_e_1_1(x_e_1)               # ActFunc X shape: torch.Size([3, 8, 900])
        # print("ActFunc X shape: "+str(x_e_1.shape))
        
        x_e_1 = self.padding_e_1_2(x_e_1)           # Padding X shape: torch.Size([3, 8, 903])
        # print("Padding X shape: "+str(x_e_1.shape))
        x_e_1 = self.conv_e_1_2(x_e_1)              # Conv X shape: torch.Size([3, 8, 900])
        # print("Conv X shape: "+str(x_e_1.shape))
        x_e_1 = self.act_e_1_2(x_e_1)               # ActFunc X shape: torch.Size([3, 8, 900])
        # print("ActFunc X shape: "+str(x_e_1.shape))
        
        x_e_p_1 = self.avgPool_e_1_1(x_e_1)         # Pool X shape: torch.Size([3, 8, 450])
        # print("Pool X shape: "+str(x_e_p_1.shape))


        # print("\n-----------E 2--------------")
        x_e_2 = self.padding_e_2_1(x_e_p_1)         # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_e_2.shape))
        x_e_2 = self.conv_e_2_1(x_e_2)              # Conv X shape: torch.Size([3, 16, 450])
        # print("Conv X shape: "+str(x_e_2.shape))
        x_e_2 = self.act_e_2_1(x_e_2)               # ActFunc X shape: torch.Size([3, 16, 450])
        # print("ActFunc X shape: "+str(x_e_2.shape))
        
        x_e_2 = self.padding_e_2_2(x_e_2)           # Padding X shape: torch.Size([3, 16, 453])
        # print("Padding X shape: "+str(x_e_2.shape))
        x_e_2 = self.conv_e_2_2(x_e_2)              # Conv X shape: torch.Size([3, 16, 450])
        # print("Conv X shape: "+str(x_e_2.shape))
        x_e_2 = self.act_e_2_2(x_e_2)               # ActFunc X shape: torch.Size([3, 16, 450])
        # print("ActFunc X shape: "+str(x_e_2.shape))

        x_e_p_2 = self.avgPool_e_2_1(x_e_2)         # Pool X shape: torch.Size([3, 16, 225])
        # print("Pool X shape: "+str(x_e_p_2.shape))


        # print("\n------------T 1-------------")
        x_t_1 = self.padding_t_1_1(x_e_p_2)         # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_t_1.shape))
        x_t_1 = self.conv_t_1_1(x_t_1)              # Conv X shape: torch.Size([3, 32, 109])
        # print("Conv X shape: "+str(x_t_1.shape))
        x_t_1 = self.act_t_1_1(x_t_1)               # ActFunc X shape: torch.Size([3, 32, 109])
        # print("ActFunc X shape: "+str(x_t_1.shape))
        
        x_t_1 = self.padding_t_1_2(x_t_1)           # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_t_1.shape))
        x_t_1 = self.conv_t_1_2(x_t_1)              # Conv X shape: torch.Size([3, 32, 106])
        # print("Conv X shape: "+str(x_t_1.shape))
        x_t_1 = self.act_t_1_2(x_t_1)               # ActFunc X shape: torch.Size([3, 32, 106])
        # print("ActFunc X shape: "+str(x_t_1.shape))


        # print("\n------------D 1------------")
        x_d_1 = self.upconv_1_1(x_t_1)              # UpConv X shape: torch.Size([3, 16, 214])
        # print("UpConv X shape: "+str(x_d_1.shape))

        x_d_1 = torch.cat([x_d_1, x_e_2], dim=1)    # 
        # print("Cat X shape: "+str(x_d_1.shape))

        x_d_1 = self.padding_e_1_1(x_d_1)           # 
        # print("Padding X shape: "+str(x_d_1.shape))
        x_d_1 = self.conv_d_1_1(x_d_1)              # 
        # print("Conv X shape: "+str(x_d_1.shape))
        x_d_1 = self.act_d_1_1(x_d_1)               # 
        # print("ActFunc X shape: "+str(x_d_1.shape))

        x_d_1 = self.padding_e_1_1(x_d_1)           # 
        # print("Padding X shape: "+str(x_d_1.shape))
        x_d_1 = self.conv_d_1_2(x_d_1)              # 
        # print("Conv X shape: "+str(x_d_1.shape))
        x_d_1 = self.act_d_1_2(x_d_1)               # 
        # print("ActFunc X shape: "+str(x_d_1.shape))


        # print("\n------------D 2------------")
        x_d_2 = self.upconv_2_1(x_d_1)              # UpConv X shape: torch.Size([3, 16, 214])
        # print("UpConv X shape: "+str(x_d_2.shape))

        x_d_2 = torch.cat([x_d_2, x_e_1], dim=1)    # 
        # print("Cat X shape: "+str(x_d_2.shape))

        x_d_2 = self.padding_e_1_1(x_d_2)           # 
        # print("Padding X shape: "+str(x_d_2.shape))
        x_d_2 = self.conv_d_2_1(x_d_2)              # 
        # print("Conv X shape: "+str(x_d_2.shape))
        x_d_2 = self.act_d_2_1(x_d_2)               # 
        # print("ActFunc X shape: "+str(x_d_2.shape))

        x_d_2 = self.padding_e_1_1(x_d_2)           # 
        # print("Padding X shape: "+str(x_d_2.shape))
        x_d_2 = self.conv_d_2_2(x_d_2)              # 
        # print("Conv X shape: "+str(x_d_2.shape))
        x_d_2 = self.act_d_2_2(x_d_2)               # 
        # print("ActFunc X shape: "+str(x_d_2.shape))


        # print("\n------------Out------------")
        # x = self.conv_out_1(x_d_2)


        x = torch.flatten(x_d_2, 1)
        x = self.output_dropout1(x)
        
        x = self.act_out_1(x)
        x = self.linear_out_1(x)
        
        
        x = self.output_dropout2(x)
        x = self.act_out_2(x)
        x = self.linear_out_2(x)

        # print("Output X shape: "+str(x.shape))

        return x

In [ ]:

class UnetBasedCNNWithDilationClassifier(nn.Module):
    
    def __init__(self, nClasses):
        super(UnetBasedCNNWithDilationClassifier, self).__init__()

        # self.input_dropout1 = nn.Dropout(p=0.2)

        # First Encode Level
        self.padding_e_1_1 = nn.CircularPad1d((1,2))
        self.conv_e_1_1 = nn.Conv1d(4, 8, kernel_size=4)
        self.act_e_1_1 = nn.ReLU()
        
        self.convd_e_1_1 = nn.Conv1d(4, 8, kernel_size=4, groups=4, dilation=2, padding=3, padding_mode="circular")
        self.actd_e_1_1 = nn.ReLU()

        self.padding_e_1_2 = nn.CircularPad1d((1,2))
        self.conv_e_1_2 = nn.Conv1d(16, 16, kernel_size=4)
        self.act_e_1_2 = nn.ReLU()

        self.avgPool_e_1_1 = nn.AvgPool1d(2, stride=2)


        # Second Encode Level        
        self.padding_e_2_1 = nn.CircularPad1d((1,2))
        self.conv_e_2_1 = nn.Conv1d(16, 32, kernel_size=4)
        self.act_e_2_1 = nn.ReLU()
        
        self.convd_e_2_1 = nn.Conv1d(16, 32, kernel_size=4, groups=4, dilation=2, padding=3, padding_mode="circular")
        self.actd_e_2_1 = nn.ReLU()
        
        self.padding_e_2_2 = nn.CircularPad1d((1,2))
        self.conv_e_2_2 = nn.Conv1d(64, 64, kernel_size=4)
        self.act_e_2_2 = nn.ReLU()

        self.avgPool_e_2_1 = nn.AvgPool1d(2, stride=2)

        
        # Transition Level
        self.padding_t_1_1 = nn.CircularPad1d((1,2))
        self.conv_t_1_1 = nn.Conv1d(64, 128, kernel_size = 4)
        self.act_t_1_1 = nn.ReLU()
        
        self.padding_t_1_2 = nn.CircularPad1d((1,2))
        self.conv_t_1_2 = nn.Conv1d(128, 128, kernel_size = 4)
        self.act_t_1_2 = nn.ReLU()


        # First Decode Level
        self.upconv_1_1 = nn.ConvTranspose1d(128, 64, kernel_size=2, stride=2)

        self.padding_d_1_1 = nn.CircularPad1d((1,2))
        self.conv_d_1_1 = nn.Conv1d(128, 64, kernel_size=4)
        self.act_d_1_1 = nn.ReLU()

        self.padding_d_1_2 = nn.CircularPad1d((1,2))
        self.conv_d_1_2 = nn.Conv1d(64, 64, kernel_size=4)
        self.act_d_1_2 = nn.ReLU()


        # Second Decode Level
        self.upconv_2_1 = nn.ConvTranspose1d(64, 32, kernel_size=2, stride=2)

        self.padding_d_2_1 = nn.CircularPad1d((1,2))
        self.conv_d_2_1 = nn.Conv1d(48, 24, kernel_size=4)
        self.act_d_2_1 = nn.ReLU()

        self.padding_d_2_2 = nn.CircularPad1d((1,2))
        self.conv_d_2_2 = nn.Conv1d(24, 24, kernel_size=4)
        self.act_d_2_2 = nn.ReLU()


        # Output Level
        self.conv_out_1 = nn.Conv1d(24, nClasses, kernel_size=1)
        
        
        # self.output_dropout1 = nn.Dropout(p=0.2)
        self.linear_out_1 = nn.Linear(21600, 43200)
        self.act_out_1 = nn.ReLU()

        
        # self.output_dropout2 = nn.Dropout(p=0.2)
        self.linear_out_2 = nn.Linear(43200, nClasses)
        self.act_out_2 = nn.ReLU()
    
    def forward(self, x):

        # print("X shape: "+str(x.shape))             # X shape: torch.Size([3, 900, 4])

        x = x.reshape(x.shape[0],4,-1)              # Reshape X shape: torch.Size([3, 4, 900])
        # print("Reshape X shape: "+str(x.shape))
        # print("\n")

        # x = self.input_dropout1(x)

        # print("\n------------E 1-------------")
        x_e_1 = self.padding_e_1_1(x)               # Padding X shape: torch.Size([3, 4, 900])
        x_e_1 = self.conv_e_1_1(x_e_1)              # Conv X shape: torch.Size([3, 8, 900])        
        x_e_1 = self.act_e_1_1(x_e_1)               # ActFunc X shape: torch.Size([3, 8, 900])        

        x_e_1_d = self.convd_e_1_1(x)
        x_e_1_d = self.actd_e_1_1(x_e_1_d)
        
        x_e_1 = torch.cat([x_e_1, x_e_1_d], dim=1)
        
        x_e_1 = self.padding_e_1_2(x_e_1)           # Padding X shape: torch.Size([3, 8, 903])
        x_e_1 = self.conv_e_1_2(x_e_1)              # Conv X shape: torch.Size([3, 8, 900])
        x_e_1 = self.act_e_1_2(x_e_1)               # ActFunc X shape: torch.Size([3, 8, 900])
        
        # print("X_e_1 shape: "+str(x_e_1.shape))
        x_e_p_1 = self.avgPool_e_1_1(x_e_1)         # Pool X shape: torch.Size([3, 8, 450])
        # print("Pool X shape: "+str(x_e_p_1.shape))

        # print("\n-----------E 2--------------")
        x_e_2 = self.padding_e_2_1(x_e_p_1)         # Padding X shape: torch.Size([3, 8, 453])
        x_e_2 = self.conv_e_2_1(x_e_2)              # Conv X shape: torch.Size([3, 16, 450])
        x_e_2 = self.act_e_2_1(x_e_2)               # ActFunc X shape: torch.Size([3, 16, 450])

        x_e_2_d = self.convd_e_2_1(x_e_p_1)
        x_e_2_d = self.actd_e_2_1(x_e_2_d)        
        x_e_2 = torch.cat([x_e_2, x_e_2_d], dim=1)
        # print("X_e_2 shape: "+str(x_e_2.shape))
        
        x_e_2 = self.padding_e_2_2(x_e_2)           # Padding X shape: torch.Size([3, 16, 453])
        # print("Padding X shape: "+str(x_e_2.shape))
        x_e_2 = self.conv_e_2_2(x_e_2)              # Conv X shape: torch.Size([3, 16, 450])
        # print("Conv X shape: "+str(x_e_2.shape))
        x_e_2 = self.act_e_2_2(x_e_2)               # ActFunc X shape: torch.Size([3, 16, 450])
        # print("ActFunc X shape: "+str(x_e_2.shape))

        x_e_p_2 = self.avgPool_e_2_1(x_e_2)         # Pool X shape: torch.Size([3, 16, 225])
        # print("Pool X shape: "+str(x_e_p_2.shape))


        # print("\n------------T 1-------------")
        x_t_1 = self.padding_t_1_1(x_e_p_2)         # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_t_1.shape))
        x_t_1 = self.conv_t_1_1(x_t_1)              # Conv X shape: torch.Size([3, 32, 109])
        # print("Conv X shape: "+str(x_t_1.shape))
        x_t_1 = self.act_t_1_1(x_t_1)               # ActFunc X shape: torch.Size([3, 32, 109])
        # print("ActFunc X shape: "+str(x_t_1.shape))
        
        x_t_1 = self.padding_t_1_2(x_t_1)           # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_t_1.shape))
        x_t_1 = self.conv_t_1_2(x_t_1)              # Conv X shape: torch.Size([3, 32, 106])
        # print("Conv X shape: "+str(x_t_1.shape))
        x_t_1 = self.act_t_1_2(x_t_1)               # ActFunc X shape: torch.Size([3, 32, 106])
        # print("ActFunc X shape: "+str(x_t_1.shape))


        # print("\n------------D 1------------")
        x_d_1 = self.upconv_1_1(x_t_1)              # UpConv X shape: torch.Size([3, 16, 214])
        # print("UpConv X shape: "+str(x_d_1.shape))

        x_d_1 = torch.cat([x_d_1, x_e_2], dim=1)    # 
        # print("Cat X shape: "+str(x_d_1.shape))

        x_d_1 = self.padding_e_1_1(x_d_1)           # 
        # print("Padding X shape: "+str(x_d_1.shape))
        x_d_1 = self.conv_d_1_1(x_d_1)              # 
        # print("Conv X shape: "+str(x_d_1.shape))
        x_d_1 = self.act_d_1_1(x_d_1)               # 
        # print("ActFunc X shape: "+str(x_d_1.shape))

        x_d_1 = self.padding_e_1_1(x_d_1)           # 
        # print("Padding X shape: "+str(x_d_1.shape))
        x_d_1 = self.conv_d_1_2(x_d_1)              # 
        # print("Conv X shape: "+str(x_d_1.shape))
        x_d_1 = self.act_d_1_2(x_d_1)               # 
        # print("ActFunc X shape: "+str(x_d_1.shape))


        # print("\n------------D 2------------")
        x_d_2 = self.upconv_2_1(x_d_1)              # UpConv X shape: torch.Size([3, 16, 214])
        # print("UpConv X shape: "+str(x_d_2.shape))

        x_d_2 = torch.cat([x_d_2, x_e_1], dim=1)    # 
        # print("Cat X shape: "+str(x_d_2.shape))

        x_d_2 = self.padding_e_1_1(x_d_2)           # 
        # print("Padding X shape: "+str(x_d_2.shape))
        x_d_2 = self.conv_d_2_1(x_d_2)              # 
        # print("Conv X shape: "+str(x_d_2.shape))
        x_d_2 = self.act_d_2_1(x_d_2)               # 
        # print("ActFunc X shape: "+str(x_d_2.shape))

        x_d_2 = self.padding_e_1_1(x_d_2)           # 
        # print("Padding X shape: "+str(x_d_2.shape))
        x_d_2 = self.conv_d_2_2(x_d_2)              # 
        # print("Conv X shape: "+str(x_d_2.shape))
        x_d_2 = self.act_d_2_2(x_d_2)               # 
        # print("ActFunc X shape: "+str(x_d_2.shape))


        # print("\n------------Out------------")
        # # x = self.conv_out_1(x_d_2)


        x = torch.flatten(x_d_2, 1)
        # print("Flatten X shape: "+str(x.shape))

        # x = self.output_dropout1(x)
        # print("Dropped X shape: "+str(x.shape))
        x = self.act_out_1(x)
        # print("Act X shape: "+str(x.shape))
        x = self.linear_out_1(x)
        # print("Linear X shape: "+str(x.shape))
        
        # x = self.output_dropout2(x)
        x = self.act_out_2(x)
        x = self.linear_out_2(x)

        # print("Output X shape: "+str(x.shape))

        return x

In [ ]:

class UnetBasedCNNWithDropoutAndDilationClassifier(nn.Module):
    
    def __init__(self, nClasses):
        super(UnetBasedCNNWithDropoutAndDilationClassifier, self).__init__()

        self.input_dropout1 = nn.Dropout(p=0.2)

        # First Encode Level
        self.padding_e_1_1 = nn.CircularPad1d((1,2))
        self.conv_e_1_1 = nn.Conv1d(4, 8, kernel_size=4)
        self.act_e_1_1 = nn.ReLU()
        
        self.convd_e_1_1 = nn.Conv1d(4, 8, kernel_size=4, groups=4, dilation=2, padding=3, padding_mode="circular")
        self.actd_e_1_1 = nn.ReLU()

        self.padding_e_1_2 = nn.CircularPad1d((1,2))
        self.conv_e_1_2 = nn.Conv1d(16, 16, kernel_size=4)
        self.act_e_1_2 = nn.ReLU()

        self.avgPool_e_1_1 = nn.AvgPool1d(2, stride=2)


        # Second Encode Level        
        self.padding_e_2_1 = nn.CircularPad1d((1,2))
        self.conv_e_2_1 = nn.Conv1d(16, 32, kernel_size=4)
        self.act_e_2_1 = nn.ReLU()
        
        self.convd_e_2_1 = nn.Conv1d(16, 32, kernel_size=4, groups=4, dilation=2, padding=3, padding_mode="circular")
        self.actd_e_2_1 = nn.ReLU()
        
        self.padding_e_2_2 = nn.CircularPad1d((1,2))
        self.conv_e_2_2 = nn.Conv1d(64, 64, kernel_size=4)
        self.act_e_2_2 = nn.ReLU()

        self.avgPool_e_2_1 = nn.AvgPool1d(2, stride=2)

        
        # Transition Level
        self.padding_t_1_1 = nn.CircularPad1d((1,2))
        self.conv_t_1_1 = nn.Conv1d(64, 128, kernel_size = 4)
        self.act_t_1_1 = nn.ReLU()
        
        self.padding_t_1_2 = nn.CircularPad1d((1,2))
        self.conv_t_1_2 = nn.Conv1d(128, 128, kernel_size = 4)
        self.act_t_1_2 = nn.ReLU()


        # First Decode Level
        self.upconv_1_1 = nn.ConvTranspose1d(128, 64, kernel_size=2, stride=2)

        self.padding_d_1_1 = nn.CircularPad1d((1,2))
        self.conv_d_1_1 = nn.Conv1d(128, 64, kernel_size=4)
        self.act_d_1_1 = nn.ReLU()

        self.padding_d_1_2 = nn.CircularPad1d((1,2))
        self.conv_d_1_2 = nn.Conv1d(64, 64, kernel_size=4)
        self.act_d_1_2 = nn.ReLU()


        # Second Decode Level
        self.upconv_2_1 = nn.ConvTranspose1d(64, 32, kernel_size=2, stride=2)

        self.padding_d_2_1 = nn.CircularPad1d((1,2))
        self.conv_d_2_1 = nn.Conv1d(48, 24, kernel_size=4)
        self.act_d_2_1 = nn.ReLU()

        self.padding_d_2_2 = nn.CircularPad1d((1,2))
        self.conv_d_2_2 = nn.Conv1d(24, 24, kernel_size=4)
        self.act_d_2_2 = nn.ReLU()


        # Output Level
        self.conv_out_1 = nn.Conv1d(24, nClasses, kernel_size=1)
        
        
        self.output_dropout1 = nn.Dropout(p=0.2)
        self.linear_out_1 = nn.Linear(21600, 43200)
        self.act_out_1 = nn.ReLU()

        
        self.output_dropout2 = nn.Dropout(p=0.2)
        self.linear_out_2 = nn.Linear(43200, nClasses)
        self.act_out_2 = nn.ReLU()
    
    def forward(self, x):

        # print("X shape: "+str(x.shape))             # X shape: torch.Size([3, 900, 4])

        x = x.reshape(x.shape[0],4,-1)              # Reshape X shape: torch.Size([3, 4, 900])
        # print("Reshape X shape: "+str(x.shape))
        # print("\n")

        x = self.input_dropout1(x)

        # print("\n------------E 1-------------")
        x_e_1 = self.padding_e_1_1(x)               # Padding X shape: torch.Size([3, 4, 900])
        x_e_1 = self.conv_e_1_1(x_e_1)              # Conv X shape: torch.Size([3, 8, 900])        
        x_e_1 = self.act_e_1_1(x_e_1)               # ActFunc X shape: torch.Size([3, 8, 900])        

        x_e_1_d = self.convd_e_1_1(x)
        x_e_1_d = self.actd_e_1_1(x_e_1_d)
        
        x_e_1 = torch.cat([x_e_1, x_e_1_d], dim=1)
        
        x_e_1 = self.padding_e_1_2(x_e_1)           # Padding X shape: torch.Size([3, 8, 903])
        x_e_1 = self.conv_e_1_2(x_e_1)              # Conv X shape: torch.Size([3, 8, 900])
        x_e_1 = self.act_e_1_2(x_e_1)               # ActFunc X shape: torch.Size([3, 8, 900])
        
        # print("X_e_1 shape: "+str(x_e_1.shape))
        x_e_p_1 = self.avgPool_e_1_1(x_e_1)         # Pool X shape: torch.Size([3, 8, 450])
        # print("Pool X shape: "+str(x_e_p_1.shape))

        # print("\n-----------E 2--------------")
        x_e_2 = self.padding_e_2_1(x_e_p_1)         # Padding X shape: torch.Size([3, 8, 453])
        x_e_2 = self.conv_e_2_1(x_e_2)              # Conv X shape: torch.Size([3, 16, 450])
        x_e_2 = self.act_e_2_1(x_e_2)               # ActFunc X shape: torch.Size([3, 16, 450])

        x_e_2_d = self.convd_e_2_1(x_e_p_1)
        x_e_2_d = self.actd_e_2_1(x_e_2_d)        
        x_e_2 = torch.cat([x_e_2, x_e_2_d], dim=1)
        # print("X_e_2 shape: "+str(x_e_2.shape))
        
        x_e_2 = self.padding_e_2_2(x_e_2)           # Padding X shape: torch.Size([3, 16, 453])
        # print("Padding X shape: "+str(x_e_2.shape))
        x_e_2 = self.conv_e_2_2(x_e_2)              # Conv X shape: torch.Size([3, 16, 450])
        # print("Conv X shape: "+str(x_e_2.shape))
        x_e_2 = self.act_e_2_2(x_e_2)               # ActFunc X shape: torch.Size([3, 16, 450])
        # print("ActFunc X shape: "+str(x_e_2.shape))

        x_e_p_2 = self.avgPool_e_2_1(x_e_2)         # Pool X shape: torch.Size([3, 16, 225])
        # print("Pool X shape: "+str(x_e_p_2.shape))


        # print("\n------------T 1-------------")
        x_t_1 = self.padding_t_1_1(x_e_p_2)         # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_t_1.shape))
        x_t_1 = self.conv_t_1_1(x_t_1)              # Conv X shape: torch.Size([3, 32, 109])
        # print("Conv X shape: "+str(x_t_1.shape))
        x_t_1 = self.act_t_1_1(x_t_1)               # ActFunc X shape: torch.Size([3, 32, 109])
        # print("ActFunc X shape: "+str(x_t_1.shape))
        
        x_t_1 = self.padding_t_1_2(x_t_1)           # Padding X shape: torch.Size([3, 8, 453])
        # print("Padding X shape: "+str(x_t_1.shape))
        x_t_1 = self.conv_t_1_2(x_t_1)              # Conv X shape: torch.Size([3, 32, 106])
        # print("Conv X shape: "+str(x_t_1.shape))
        x_t_1 = self.act_t_1_2(x_t_1)               # ActFunc X shape: torch.Size([3, 32, 106])
        # print("ActFunc X shape: "+str(x_t_1.shape))


        # print("\n------------D 1------------")
        x_d_1 = self.upconv_1_1(x_t_1)              # UpConv X shape: torch.Size([3, 16, 214])
        # print("UpConv X shape: "+str(x_d_1.shape))

        x_d_1 = torch.cat([x_d_1, x_e_2], dim=1)    # 
        # print("Cat X shape: "+str(x_d_1.shape))

        x_d_1 = self.padding_e_1_1(x_d_1)           # 
        # print("Padding X shape: "+str(x_d_1.shape))
        x_d_1 = self.conv_d_1_1(x_d_1)              # 
        # print("Conv X shape: "+str(x_d_1.shape))
        x_d_1 = self.act_d_1_1(x_d_1)               # 
        # print("ActFunc X shape: "+str(x_d_1.shape))

        x_d_1 = self.padding_e_1_1(x_d_1)           # 
        # print("Padding X shape: "+str(x_d_1.shape))
        x_d_1 = self.conv_d_1_2(x_d_1)              # 
        # print("Conv X shape: "+str(x_d_1.shape))
        x_d_1 = self.act_d_1_2(x_d_1)               # 
        # print("ActFunc X shape: "+str(x_d_1.shape))


        # print("\n------------D 2------------")
        x_d_2 = self.upconv_2_1(x_d_1)              # UpConv X shape: torch.Size([3, 16, 214])
        # print("UpConv X shape: "+str(x_d_2.shape))

        x_d_2 = torch.cat([x_d_2, x_e_1], dim=1)    # 
        # print("Cat X shape: "+str(x_d_2.shape))

        x_d_2 = self.padding_e_1_1(x_d_2)           # 
        # print("Padding X shape: "+str(x_d_2.shape))
        x_d_2 = self.conv_d_2_1(x_d_2)              # 
        # print("Conv X shape: "+str(x_d_2.shape))
        x_d_2 = self.act_d_2_1(x_d_2)               # 
        # print("ActFunc X shape: "+str(x_d_2.shape))

        x_d_2 = self.padding_e_1_1(x_d_2)           # 
        # print("Padding X shape: "+str(x_d_2.shape))
        x_d_2 = self.conv_d_2_2(x_d_2)              # 
        # print("Conv X shape: "+str(x_d_2.shape))
        x_d_2 = self.act_d_2_2(x_d_2)               # 
        # print("ActFunc X shape: "+str(x_d_2.shape))


        # print("\n------------Out------------")
        # # x = self.conv_out_1(x_d_2)


        x = torch.flatten(x_d_2, 1)
        # print("Flatten X shape: "+str(x.shape))

        x = self.output_dropout1(x)
        # print("Dropped X shape: "+str(x.shape))
        x = self.act_out_1(x)
        # print("Act X shape: "+str(x.shape))
        x = self.linear_out_1(x)
        # print("Linear X shape: "+str(x.shape))
        
        x = self.output_dropout2(x)
        x = self.act_out_2(x)
        x = self.linear_out_2(x)

        # print("Output X shape: "+str(x.shape))

        return x

## Multi Level

In [ ]:
class MultilevelSimplestCNNClassifier(nn.Module):
    
    def __init__(self, nClasses, nPreviousClasses):
        super(MultilevelSimplestCNNClassifier, self).__init__()

        print("nClasses: "+str(nClasses))
        print("nPreviousClasses: "+str(nPreviousClasses))

        self.padding1 = nn.CircularPad1d((1,2))
        self.conv1 = nn.Conv1d(4, 8, kernel_size=4)
        self.adAvgPool1 = nn.AdaptiveAvgPool1d(450)

        self.padding2 = nn.CircularPad1d((1,2))
        self.conv2 = nn.Conv1d(8, 32, kernel_size=4)
        self.adAvgPool2 = nn.AdaptiveAvgPool1d(225)

        self.padding3 = nn.CircularPad1d((1,2))
        self.conv3 = nn.Conv1d(32, 128, kernel_size=4)
        self.adAvgPool3 = nn.AdaptiveAvgPool1d(225)


        self.act4 = nn.ReLU()

        self.linear1 = nn.Linear(28800, 28800*2)
        self.linear2 = nn.Linear((28800*2)+nPreviousClasses, nClasses)
    
    def forward(self, x, px):

        x = torch.movedim(x, -1, -2)

        x = self.conv1(self.padding1(x))
        x = self.adAvgPool1(x)

        
        x = self.conv2(self.padding2(x))
        x = self.adAvgPool2(x)

        
        x = self.conv3(self.padding3(x))
        x = self.adAvgPool3(x)
        
        
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = self.act4(x)
        
        x = torch.cat([x, px], dim=1)

        x = self.linear2(x)

        return x

In [ ]:
class MultilevelSimplestCNNClassifier2(nn.Module):
    
    def __init__(self, nClasses, nPreviousClasses):
        super(MultilevelSimplestCNNClassifier2, self).__init__()

        print("nClasses: "+str(nClasses))
        print("nPreviousClasses: "+str(nPreviousClasses))

        self.padding1 = nn.CircularPad1d((1,2))
        self.conv1 = nn.Conv1d(4, 8, kernel_size=4)
        self.adAvgPool1 = nn.AdaptiveAvgPool1d(450)

        self.padding2 = nn.CircularPad1d((1,2))
        self.conv2 = nn.Conv1d(8, 32, kernel_size=4)
        self.adAvgPool2 = nn.AdaptiveAvgPool1d(225)

        self.padding3 = nn.CircularPad1d((1,2))
        self.conv3 = nn.Conv1d(32, 128, kernel_size=4)
        self.adAvgPool3 = nn.AdaptiveAvgPool1d(225)


        self.act4 = nn.ReLU()

        self.linear1 = nn.Linear(28800, 28800)
        self.linear2 = nn.Linear((28800)+nPreviousClasses, 14400)
        
        self.act5 = nn.ReLU()

        self.linear3 = nn.Linear((14400)+nPreviousClasses, nClasses)
    
    def forward(self, x, px):

        x = torch.movedim(x, -1, -2)

        x = self.conv1(self.padding1(x))
        x = self.adAvgPool1(x)

        
        x = self.conv2(self.padding2(x))
        x = self.adAvgPool2(x)

        
        x = self.conv3(self.padding3(x))
        x = self.adAvgPool3(x)
        
        
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = self.act4(x)
        
        x = torch.cat([x, px], dim=1)

        x = self.linear2(x)
        x = self.act5(x)
        
        x = torch.cat([x, px], dim=1)

        x = self.linear3(x)

        return x

In [ ]:
class MultilevelSimplestCNNClassifier3(nn.Module):
    
    def __init__(self, nClasses, nPreviousClasses):
        super(MultilevelSimplestCNNClassifier3, self).__init__()

        print("nClasses: "+str(nClasses))
        print("nPreviousClasses: "+str(nPreviousClasses))

        self.padding1 = nn.CircularPad1d((1,2))
        self.conv1 = nn.Conv1d(4, 8, kernel_size=4)
        self.adAvgPool1 = nn.AdaptiveAvgPool1d(450)

        self.padding2 = nn.CircularPad1d((1,2))
        self.conv2 = nn.Conv1d(8, 32, kernel_size=4)
        self.adAvgPool2 = nn.AdaptiveAvgPool1d(225)

        self.padding3 = nn.CircularPad1d((1,2))
        self.conv3 = nn.Conv1d(32, 128, kernel_size=4)
        self.adAvgPool3 = nn.AdaptiveAvgPool1d(225)


        self.linear1 = nn.Linear(28800, 14400)
        self.act4 = nn.ReLU()
        self.linear2 = nn.Linear((14400)+nPreviousClasses, (7200+nClasses))
        
        self.act5 = nn.ReLU()

        self.linear3 = nn.Linear((7200+nClasses)+nPreviousClasses, nClasses)
    
    def forward(self, x, px):

        x = torch.movedim(x, -1, -2)

        x = self.conv1(self.padding1(x))
        x = self.adAvgPool1(x)

        
        x = self.conv2(self.padding2(x))
        x = self.adAvgPool2(x)

        
        x = self.conv3(self.padding3(x))
        x = self.adAvgPool3(x)
        
        
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = self.act4(x)
        
        x = torch.cat([x, px], dim=1)

        x = self.linear2(x)
        x = self.act5(x)
        
        x = torch.cat([x, px], dim=1)

        x = self.linear3(x)

        return x

In [ ]:
class MultilevelSimplestCNNClassifier4(nn.Module):
    
    def __init__(self, nClasses, nPreviousClasses):
        super(MultilevelSimplestCNNClassifier4, self).__init__()

        print("nClasses: "+str(nClasses))
        print("nPreviousClasses: "+str(nPreviousClasses))

        self.padding1 = nn.CircularPad1d((1,2))
        self.conv1 = nn.Conv1d(4, 8, kernel_size=4)
        self.adAvgPool1 = nn.AdaptiveAvgPool1d(450)

        self.padding2 = nn.CircularPad1d((1,2))
        self.conv2 = nn.Conv1d(8, 32, kernel_size=4)
        self.adAvgPool2 = nn.AdaptiveAvgPool1d(225)

        self.padding3 = nn.CircularPad1d((1,2))
        self.conv3 = nn.Conv1d(32, 128, kernel_size=4)
        self.adAvgPool3 = nn.AdaptiveAvgPool1d(225)


        self.act4 = nn.ReLU()

        self.linear1 = nn.Linear(28800, 7200)
        self.linear2 = nn.Linear((7200)+nPreviousClasses, (nClasses*2))
        
        self.act5 = nn.ReLU()

        self.linear3 = nn.Linear((nClasses*2)+nPreviousClasses, nClasses)
    
    def forward(self, x, px):

        x = torch.movedim(x, -1, -2)

        x = self.conv1(self.padding1(x))
        x = self.adAvgPool1(x)

        
        x = self.conv2(self.padding2(x))
        x = self.adAvgPool2(x)

        
        x = self.conv3(self.padding3(x))
        x = self.adAvgPool3(x)
        
        
        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = self.act4(x)
        
        x = torch.cat([x, px], dim=1)

        x = self.linear2(x)
        x = self.act5(x)
        
        x = torch.cat([x, px], dim=1)

        x = self.linear3(x)

        return x

## Test Params

In [ ]:
_levels_ = [
    # "genus",
    # "species",
    # "order", 
    "family", 
    "class", 
]

In [ ]:
_batch_sizes_ = [
    # 64,
    # 128,
    # 256,
    # 512,
    # 2048,
    "dynamic"
]

In [ ]:
_epochs_ = [
    # 2,
    # 5,
    # 30,
    # 50,
    # 100,
    # 150,
    200,
    # 500
]

In [ ]:
_models_list_ = [
    MultilevelSimplestCNNClassifier4,
    MultilevelSimplestCNNClassifier3,
    MultilevelSimplestCNNClassifier2,
    MultilevelSimplestCNNClassifier,
    # SimplestCNNClassifier,
    # SimpleCNNClassifier,
    # SimpleCNNWithDropoutClassifier,
    # UnetBasedCNNClassifier,
    # UnetBasedCNNWithDilationClassifier,
    # UnetBasedCNNWithDropoutClassifier,
    # UnetBasedCNNWithDropoutAndDilationClassifier,
    # BaseCNNClassifier, 
]

In [ ]:
_loss_functions_ = {
    "CrossEntropyLoss":{
        "function":nn.CrossEntropyLoss,
        "params":{},
        "function_params":{}
    },
}

In [ ]:
_learning_rates_ = [
    # 1e-2,
    # 5e-2,
    # 1e-3,
    # 5e-3,
    # 1e-4,
    5e-4,
    5e-5,
]

In [ ]:
_optimizers_ = [
    {
        "optim":torch.optim.AdamW,
        "params":{
            "weight_decay":1e-2,
            "amsgrad":True
        }
    },
]

In [ ]:
hiperparams = {
    "levels": _levels_,
    "batch_size": _batch_sizes_,
    "epochs": _epochs_,
    "model": _models_list_,
    "loss_function": _loss_functions_,
    "learning_rate": _learning_rates_,
    "optimizer": _optimizers_    
}

In [ ]:
hiperparams

## Batch Execution

In [ ]:
def Train_Test(
        model, 
        loss_fn, 
        optimizer, 
        epochs, 
        learning_rate, 
        batch_size, 
        train_data,
        test_data,
        id="", 
        ):
    
    print("Model: \t\t\t"+(model._get_name() if not model._get_name() == "OptimizedModule" else model.__dict__["_modules"]["_orig_mod"].__class__.__name__))
    print("  Loss Func.: \t\t"+loss_fn._get_name())
    print("  Optimizer: \t\t"+type(optimizer).__name__)
    print("  Epochs: \t\t"+str(epochs))
    print("  Learning Rate: \t"+str(learning_rate))

    print("\nModel Arch: ")
    print(str(model))
    print("\n\n\n")

    
    if torch.cuda.get_device_capability() < (7, 0):
        print("Exiting because torch.compile is not supported on this device.")
        import sys
        sys.exit(0)

    epochs_results = []
    current = {
        "model":(model._get_name() if not model._get_name() == "OptimizedModule" else model.__dict__["_modules"]["_orig_mod"].__class__.__name__),
        "loss_function":loss_fn._get_name(),
        "epoch":None,
        "learning_rate":learning_rate,
        "batch_size":None,
        "train_size":None,
        "test_size":None,
        "optimizer":type(optimizer).__name__,
        "train_acc":None,
        "train_loss":None,
        "test_acc":None,
        "test_loss":None,
    }

    if batch_size == "dynamic":
        bss = [1000, 500, 250, 250, 500, 1000]
    else:
        bss = [batch_size]
    if len(bss) > epochs:
        bss = bss[0:epochs]
    print("Batch Sizes List: "+str(bss))
    batch_lim = int(epochs/len(bss))
    
    
    t_start = time.time()
    
    best = {
        "epoch":0,
        "train_acc":0,
        "train_loss":10000000,
        "test_acc":0,
        "test_loss":10000000,
    }
    
    train_loader = None
    test_loader = None
    
    # Epochs
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}\n-------------------------------")

        if epoch%batch_lim == 0 and len(bss) > 0:
            if train_loader:
                del train_loader
            if test_loader:
                del test_loader

            batch_size = bss.pop(0)
            train_loader, test_loader = loaders_generator(train_data, test_data, batch_size)

        print("Batch Size: "+str(batch_size))

        # Train --------------------------------------------------------------------------
                    
        #@torch.compile(fullgraph=False)
        fn = torch.compile(optimizer.step)

        model.train()
        train_loss = 0
        train_acc = 0

        for batch, (X, prevy, y) in enumerate(train_loader):
            optimizer.zero_grad()

            # Compute prediction and loss
            pred = model(X, prevy)
            loss = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            fn()

            # Update results
            train_loss += loss.item()
            train_acc += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

        # Train results
        train_loss /= len(train_loader)
        train_acc /= len(train_loader.dataset)
        print(f"Train: \n Accuracy: {(100*train_acc):>0.1f}%, Avg loss: {train_loss:>8f} \n")


        # Test
        model.eval()
        test_loss = 0
        test_acc = 0

        with torch.no_grad():
            for X, prevy, y in test_loader:
                pred = model(X, prevy)
                test_loss += loss_fn(pred, y).item()
                test_acc += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

        # Test results
        test_loss /= len(test_loader)
        test_acc /= len(test_loader.dataset)
        print(f"Test: \n Accuracy: {(100*test_acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")


        # Update Results
        if best["test_acc"] < test_acc or (best["test_acc"] == test_acc and best["train_acc"] < train_acc):
            best["epoch"] = epoch+1
            best["test_acc"] = test_acc
            best["test_loss"] = test_loss
            best["train_acc"] = train_acc
            best["train_loss"] = train_loss

            # if test_acc > 0.5:
                # torch.save(model.state_dict(), "/media/stark/Models/Gustavo/"+train_data.level+"/"+str(id)+"_"+current["model"]+".pth")
                                
            
        
        current["epoch"] = epoch+1
        current["batch_size"] = batch_size
        current["train_size"] = train_loader.dataset.__len__()
        current["test_size"] = test_loader.dataset.__len__()
        current["train_acc"] = train_acc
        current["train_loss"] = train_loss
        current["test_acc"] = test_acc
        current["test_loss"] = test_loss

    
        epochs_results.append(current.copy())

    pd.DataFrame(epochs_results).to_csv("./results/epochs/"+str(id)+"__"+current["model"]+"_train_test.csv")
    
    print("\n\n")
    print(f"Best Epoch:{best['epoch']} \n\tAccuracy: {(100*best['test_acc']):>0.1f}%, Avg loss: {best['test_loss']:>8f} \n")
    print("Train and Test execution time: "+str(format(time.time()-t_start, '.4f'))+"s")
    print("Done!")

    return best

In [ ]:
_model_ = None
_lossfunction_ = None
_optimizer_ = None

def clear():
    global _model_, _lossfunction_, _optimizer_
    
    torch.cuda.empty_cache()

    torch.compiler.reset()
    torch._dynamo.reset()

    if _model_:
        del _model_
        _model_ = None
    if _lossfunction_:
        del _lossfunction_
        _lossfunction_ = None
    if _optimizer_:
        del _optimizer_
        _optimizer_ = None
    torch.cuda.empty_cache()

results = []
current = {}

id = 0
time_id = str(int(time.time()))
print("Time ID: "+str(time_id))

for level in hiperparams["levels"]:
    clear()

    train_data = pd.read_csv("../new_data/StratifiedSplit/"+level+"/train_dataset.csv")
    test_data = pd.read_csv("../new_data/StratifiedSplit/"+level+"/test_dataset.csv")
    print(level)
    print(train_data.shape)
    print(test_data.shape)

    dataset = MultilevelSequenceDataset(
        train=train_data, 
        test=test_data, 
        level=level)


    for batch_size in hiperparams["batch_size"]:
        for epochs in hiperparams["epochs"]:
            for model in hiperparams["model"]:
                for loss_function_name, loss_function in hiperparams["loss_function"].items():
                    for learning_rate in hiperparams["learning_rate"]:
                        for optimizer in hiperparams["optimizer"]:
                            
                            optim = optimizer["optim"]
                            optim_params = optimizer["params"] if "params" in optimizer.keys() else {}

                            current = {
                                    "id": id,
                                    "start_time":time.time(),
                                    "end_time": None,
                                    "level": level,
                                    "batch_size": batch_size,
                                    "epochs": epochs,
                                    "model": model.__name__,
                                    "loss_function": loss_function_name+" ("+str(loss_function["function"])+")",
                                    "learning_rate": learning_rate,
                                    "optimizer": optim.__name__+" (params: "+str(optim_params)+")",
                                    "obs": "9:1",
                                    "error": None
                                }


                            try:                                
                                clear()
                                torch.set_float32_matmul_precision('high')
                                
                                _model_ = torch.compile(model(dataset.encoded_labels.shape[1], dataset.previous_encoded_labels.shape[1]))
                                _lossfunction_ = loss_function["function"](**{func:params[0](*params[1:]) for func,params in loss_function["function_params"].items()})
                                _optimizer_ = optim(_model_.parameters(), lr=learning_rate, **optim_params)


                                # ---- Run ----
                                result = Train_Test(
                                    model=_model_,
                                    loss_fn=_lossfunction_,
                                    optimizer=_optimizer_,
                                    epochs=epochs,
                                    learning_rate=learning_rate,
                                    batch_size=batch_size,
                                    train_data=dataset,
                                    test_data=dataset.get_test(),
                                    id=time_id+"_"+str(id),
                                    )
                                    
                                current["end_time"] = time.time()
                                current["best_epoch"] = result["epoch"]
                                current["train_acc_best_epoch"] = result["train_acc"]
                                current["train_loss_best_epoch"] = result["train_loss"]
                                current["test_acc_best_epoch"] = result["test_acc"]
                                current["test_loss_best_epoch"] = result["test_loss"]

                                
                                clear()                                
                                
                            except Exception as e:
                                print(e)
                                current["error"] = str(e)
                            
                            results.append(current)
                            pd.DataFrame(results).to_csv("./results/summarized/"+str(time_id)+"_models_train_test_"+str(len(results))+".csv")
                            
                            id = id+1

clear()



In [ ]:
results